In [0]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=False)
!ls "/content/gdrive/My Drive/Colab Notebooks"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'2_Movie sentiment neural network.ipynb'
'2 ways to struture your nn in keras.ipynb'
'3_Movie sentiment neural network.ipynb'
 blend1.csv
 blend_conv_conv_6.csv
 blend.csv
 blend_gru_lstm_conv_7.csv
 blend_gru_lstm_conv.csv
 blend.gsheet
 blend_lstm_lstm_conv_5.csv
 blend_valid.csv
'Copy of movie_analysis_6'
'Custom VGG16.ipynb'
'keras tokenizer.ipynb'
 movie_analysis_2
 movie_analysis_3
 movie_analysis_4
 movie_analysis_5
 movie_analysis_6
 movie_analysis_7
 movie_analysis.ipynb
'Movie sentiment neural network.ipynb'
 project
 project_2.ipynb
 sentimental_movie_analysis
 sentiment-analysis-on-movie-reviews
 Untitled0.ipynb
 Untitled1.ipynb
 VGG16.ipynb


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import gensim
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

Getting All the Data

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews/train.tsv', sep="\t")
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews/test.tsv', sep="\t")
sub = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews/sampleSubmission.csv', sep=",")

Catagories variation in dataset

In [0]:
train.groupby(train.Sentiment)['Sentiment'].count()

Sentiment
0     7072
1    27273
2    79582
3    32927
4     9206
Name: Sentiment, dtype: int64

Balancing the sample in data set, using Stratify for train and test split

In [0]:
train_no_2=train[~(train["Sentiment"]==2)]
train_with_2=train[(train["Sentiment"]==2)]
train_with_2 = train_with_2.sample(28000)
frames=[train_no_2,train_with_2]
train=pd.concat(frames)
train, val_data = train_test_split(train, test_size = 0.2, stratify=train['Sentiment'], random_state=200)

In [0]:
train.groupby(train.Sentiment)['Sentiment'].count()

Sentiment
0     5658
1    21818
2    22400
3    26341
4     7365
Name: Sentiment, dtype: int64

In [0]:
y = train['Sentiment']

Tokenizer for Phrase to text and sequence

In [0]:
tk = Tokenizer(lower = True, filters='')

In [0]:
train_tokenized = tk.texts_to_sequences(train['Phrase'])
test_tokenized = tk.texts_to_sequences(test['Phrase'])

In [0]:
max_len = 50
X_train = pad_sequences(train_tokenized, maxlen = max_len)
X_test = pad_sequences(test_tokenized, maxlen = max_len)

In [0]:
embedding_path = "/content/gdrive/My Drive/Colab Notebooks/project/GoogleNews-vectors-negative300.bin"

In [0]:
embed_size = 300
max_features = 30000

Importing Google Pretrained Gensim Model

In [0]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format(embedding_path, binary=True)

In [0]:
NUM_WORDS=20000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(list(train['Phrase'].values))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 15161 unique tokens.


Generating Embedding Matrix

In [0]:
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

One-Hot Encoder

In [0]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(y.values.reshape(-1, 1))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [0]:
def build_model1(lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    file_path = "my_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                  save_best_only = True, mode = "min")
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
    
    inp = Input(shape = (max_len,))
    x = Embedding(15162, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x1 = SpatialDropout1D(spatial_dr)(x)

    x_gru = Bidirectional(CuDNNLSTM(units, return_sequences = True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool1_gru = GlobalAveragePooling1D()(x1)
    max_pool1_gru = GlobalMaxPooling1D()(x1)

    x_lstm = Bidirectional(CuDNNLSTM (units, return_sequences = True))(x1)
    x1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool1_lstm = GlobalAveragePooling1D()(x1)
    max_pool1_lstm = GlobalMaxPooling1D()(x1)
    
    
    x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool1_lstm, max_pool1_lstm])
    #x = BatchNormalization()(x_gru)
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(dense_units, activation='relu') (x))
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(int(dense_units / 2), activation='relu') (x))
    x = Dense(5, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, y_ohe, batch_size = 128, epochs = 20, validation_split=0.2, 
                        verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model

In [0]:
model1 = build_model1(lr = 1e-3, lr_d = 1e-10, units = 64, spatial_dr = 0.3, kernel_size1=3, kernel_size2=2, dense_units=32, dr=0.1, conv_size=32)

Train on 66865 samples, validate on 16717 samples
Epoch 1/20
66865/66865 [==============================] - 44s 665us/step - loss: 0.4975 - acc: 0.7654 - val_loss: 0.4502 - val_acc: 0.8009

Epoch 00001: val_loss improved from inf to 0.45018, saving model to my_model.hdf5
Epoch 2/20
66865/66865 [==============================] - 36s 539us/step - loss: 0.4560 - acc: 0.7988 - val_loss: 0.4452 - val_acc: 0.8014

Epoch 00002: val_loss improved from 0.45018 to 0.44525, saving model to my_model.hdf5
Epoch 3/20
66865/66865 [==============================] - 30s 449us/step - loss: 0.4456 - acc: 0.8009 - val_loss: 0.4360 - val_acc: 0.8018

Epoch 00003: val_loss improved from 0.44525 to 0.43602, saving model to my_model.hdf5
Epoch 4/20
66865/66865 [==============================] - 31s 470us/step - loss: 0.4364 - acc: 0.8027 - val_loss: 0.4284 - val_acc: 0.8036

Epoch 00004: val_loss improved from 0.43602 to 0.42840, saving model to my_model.hdf5
Epoch 5/20
66865/66865 [==========================

In [0]:
model2 = build_model1(lr = 1e-3, lr_d = 1e-10, units = 128, spatial_dr = 0.5, kernel_size1=3, kernel_size2=2, dense_units=64, dr=0.2, conv_size=32)

Train on 66865 samples, validate on 16717 samples
Epoch 1/20
66865/66865 [==============================] - 47s 703us/step - loss: 0.4881 - acc: 0.7788 - val_loss: 0.4534 - val_acc: 0.8000

Epoch 00001: val_loss improved from inf to 0.45335, saving model to my_model.hdf5
Epoch 2/20
66865/66865 [==============================] - 39s 590us/step - loss: 0.4563 - acc: 0.7986 - val_loss: 0.4595 - val_acc: 0.7971

Epoch 00002: val_loss did not improve from 0.45335
Epoch 3/20
66865/66865 [==============================] - 39s 577us/step - loss: 0.4495 - acc: 0.8003 - val_loss: 0.4428 - val_acc: 0.8011

Epoch 00003: val_loss improved from 0.45335 to 0.44284, saving model to my_model.hdf5
Epoch 4/20
66865/66865 [==============================] - 39s 578us/step - loss: 0.4445 - acc: 0.8008 - val_loss: 0.4491 - val_acc: 0.8007

Epoch 00004: val_loss did not improve from 0.44284
Epoch 5/20
66865/66865 [==============================] - 39s 579us/step - loss: 0.4392 - acc: 0.8016 - val_loss: 0.4349

In [0]:
def build_model2(lr=0.0, lr_d=0.0, units=0, spatial_dr=0.0, kernel_size1=3, kernel_size2=2, dense_units=128, dr=0.1, conv_size=32):
    file_path = "my_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                  save_best_only = True, mode = "min")
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)

    inp = Input(shape = (max_len,))
    x = Embedding(15162, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x1 = SpatialDropout1D(spatial_dr)(x)

    x_gru = Bidirectional(CuDNNLSTM(units, return_sequences = True))(x1)
    x_lstm = Bidirectional(CuDNNLSTM (units, return_sequences = True))(x_gru)
    
    x_conv1 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_gru)
    avg_pool1_gru = GlobalAveragePooling1D()(x_conv1)
    max_pool1_gru = GlobalMaxPooling1D()(x_conv1)

    x_conv3 = Conv1D(conv_size, kernel_size=kernel_size1, padding='valid', kernel_initializer='he_uniform')(x_lstm)
    avg_pool1_lstm = GlobalAveragePooling1D()(x_conv3)
    max_pool1_lstm = GlobalMaxPooling1D()(x_conv3)
    
    
    x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool1_lstm, max_pool1_lstm])
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(dense_units, activation='relu') (x))
    x = BatchNormalization()(x)
    x = Dropout(dr)(Dense(int(dense_units / 2), activation='relu') (x))
    x = Dense(5, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, y_ohe, batch_size = 128, epochs = 20, validation_split=0.2, 
                        verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model

In [0]:
model3 = build_model2(lr = 1e-4, lr_d = 0, units = 64, spatial_dr = 0.5, kernel_size1=4, kernel_size2=3, dense_units=32, dr=0.1, conv_size=32)

Train on 66865 samples, validate on 16717 samples
Epoch 1/20
66865/66865 [==============================] - 45s 669us/step - loss: 0.5602 - acc: 0.7401 - val_loss: 0.4738 - val_acc: 0.7985

Epoch 00001: val_loss improved from inf to 0.47379, saving model to my_model.hdf5
Epoch 2/20
66865/66865 [==============================] - 41s 620us/step - loss: 0.4893 - acc: 0.7846 - val_loss: 0.4565 - val_acc: 0.7997

Epoch 00002: val_loss improved from 0.47379 to 0.45654, saving model to my_model.hdf5
Epoch 3/20
66865/66865 [==============================] - 31s 465us/step - loss: 0.4752 - acc: 0.7887 - val_loss: 0.4521 - val_acc: 0.8000

Epoch 00003: val_loss improved from 0.45654 to 0.45213, saving model to my_model.hdf5
Epoch 4/20
66865/66865 [==============================] - 29s 441us/step - loss: 0.4691 - acc: 0.7904 - val_loss: 0.4505 - val_acc: 0.8008

Epoch 00004: val_loss improved from 0.45213 to 0.45047, saving model to my_model.hdf5
Epoch 5/20
66865/66865 [==========================

In [0]:
model4 = build_model2(lr = 1e-3, lr_d = 0, units = 64, spatial_dr = 0.5, kernel_size1=3, kernel_size2=3, dense_units=64, dr=0.3, conv_size=32)

Train on 66865 samples, validate on 16717 samples
Epoch 1/20
66865/66865 [==============================] - 48s 720us/step - loss: 0.5110 - acc: 0.7645 - val_loss: 0.4501 - val_acc: 0.7999

Epoch 00001: val_loss improved from inf to 0.45013, saving model to my_model.hdf5
Epoch 2/20
66865/66865 [==============================] - 41s 618us/step - loss: 0.4607 - acc: 0.7987 - val_loss: 0.4497 - val_acc: 0.8004

Epoch 00002: val_loss improved from 0.45013 to 0.44966, saving model to my_model.hdf5
Epoch 3/20
66865/66865 [==============================] - 33s 499us/step - loss: 0.4520 - acc: 0.8000 - val_loss: 0.4436 - val_acc: 0.8017

Epoch 00003: val_loss improved from 0.44966 to 0.44358, saving model to my_model.hdf5
Epoch 4/20
66865/66865 [==============================] - 31s 463us/step - loss: 0.4480 - acc: 0.8005 - val_loss: 0.4419 - val_acc: 0.8026

Epoch 00004: val_loss improved from 0.44358 to 0.44194, saving model to my_model.hdf5
Epoch 5/20
66865/66865 [==========================

In [0]:
model5 = build_model2(lr = 1e-3, lr_d = 1e-7, units = 64, spatial_dr = 0.3, kernel_size1=3, kernel_size2=3, dense_units=64, dr=0.4, conv_size=64)

Train on 66865 samples, validate on 16717 samples
Epoch 1/20
66865/66865 [==============================] - 40s 592us/step - loss: 0.5309 - acc: 0.7487 - val_loss: 0.4568 - val_acc: 0.8000

Epoch 00001: val_loss improved from inf to 0.45681, saving model to my_model.hdf5
Epoch 2/20
66865/66865 [==============================] - 32s 483us/step - loss: 0.4633 - acc: 0.7988 - val_loss: 0.4469 - val_acc: 0.8000

Epoch 00002: val_loss improved from 0.45681 to 0.44689, saving model to my_model.hdf5
Epoch 3/20
66865/66865 [==============================] - 31s 462us/step - loss: 0.4539 - acc: 0.8001 - val_loss: 0.4509 - val_acc: 0.8004

Epoch 00003: val_loss did not improve from 0.44689
Epoch 4/20
66865/66865 [==============================] - 30s 454us/step - loss: 0.4473 - acc: 0.8008 - val_loss: 0.4426 - val_acc: 0.8029

Epoch 00004: val_loss improved from 0.44689 to 0.44258, saving model to my_model.hdf5
Epoch 5/20
66865/66865 [==============================] - 31s 469us/step - loss: 0.43

In [0]:
pred1 = model1.predict(X_test, batch_size = 1024, verbose = 1)
pred = pred1
pred2 = model2.predict(X_test, batch_size = 1024, verbose = 1)
pred += pred2
pred3 = model3.predict(X_test, batch_size = 1024, verbose = 1)
pred += pred3
pred4 = model4.predict(X_test, batch_size = 1024, verbose = 1)
pred += pred4
pred5 = model5.predict(X_test, batch_size = 1024, verbose = 1)
pred += pred5

66292/66292 [==============================] - 6s 87us/step


In [0]:
predictions = np.round(np.argmax(pred, axis=1)).astype(int)
sub['Sentiment'] = predictions
sub.to_csv("/content/gdrive/My Drive/Colab Notebooks/lstm_lstm_conv.csv", index=False)

In [0]:
X_test.size

3314600

In [0]:
result_df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lstm_lstm_conv.csv")

In [0]:
result_df

,PhraseId,Sentiment
0,156061,3
1,156062,3
2,156063,2
3,156064,3
4,156065,2
5,156066,2
6,156067,2
7,156068,2
8,156069,2
9,156070,2
